# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from math import *

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Rocketz\Documents\GitHub\Projeto_1-Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Tesla_pronto.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,"@danielarpm tesla, sem hesitação.",0
1,"@odlareg23 novo mundo, meu amigo. veja a amazo...",1
2,"pra namorar comigo, tem que pelo menos amar o ...",0
3,eu sou fanatica pelo tesla,0
4,"cara que baguiu chave man, tesla era um cara i...",0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,@pjfdf @blnarymlke @_tuaregue_ isso de certeza...,1
1,caralho nikolas tesla era mo pau no cu,0
2,@tiago86castro olha que qnd andar de cajata já...,0
3,novo status: pessoas que visitaram meu perfil ...,0
4,tesla: banco americano prevê lucro maior e car...,1


In [6]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;"\n"()''"",_%$\|/,<>]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    #filtro que remove as menções à usuários com @
    filtro = filter(lambda x:x[0]!='@', text_subbed.split())
    juncao = " ".join(filter(lambda x:x[0]!='@', text_subbed.split()))
    
    return juncao

In [29]:
train['Treinamento'] = train['Treinamento'].apply(cleanup)
train['Treinamento'] = train['Treinamento'].str.lower()

In [30]:
train['Relevancia'] = train['Relevancia'].astype('category')
train['Relevancia'].cat.categories = ['Irrelevante','Relevante']

___
## Classificador automático de sentimento


In [31]:
relevante = train['Relevancia'] == 'Relevante'
irrelevante = train['Relevancia'] == 'Irrelevante'

In [32]:
df_relevante = train.loc[relevante, :]
df_irrelevante = train.loc[irrelevante, :]

In [33]:
tweets_relevantes = []
lista_r = []
for r in df_relevante['Treinamento']:
    tweets_relevantes.append(r.split(' '))
for rl in tweets_relevantes:
    lista_r += rl
tweets_irrelevantes = []
lista_i = []
for i in df_irrelevante['Treinamento']:
    tweets_irrelevantes.append(i.split(' '))
for il in tweets_irrelevantes:
    lista_i += il

In [34]:
serie_relevante = pd.Series(lista_r)
serie_irrelevante = pd.Series(lista_i)

In [79]:
alpha = 1
v = 1e4

In [80]:
freq_ab_r = serie_relevante.value_counts() 
freq_re_r = (freq_ab_r + alpha)/(freq_ab_r.sum() + (alpha*v))

In [81]:
freq_ab_i = serie_irrelevante.value_counts() 
freq_re_i = (freq_ab_i + alpha)/(freq_ab_i.sum() + (alpha*v)) 

Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [82]:
p_r = len(tweets_relevantes) / len(train["Treinamento"])
p_i = len(tweets_irrelevantes) / len(train["Treinamento"])

In [83]:
def datawitter(frase):
    
    
    prob_frase_dado_r = 1
    prob_frase_dado_i = 1
   
    
    frase = frase.split(' ')
    
    for palavra in frase:
        
        if palavra in freq_re_r:
            prob_frase_dado_r *= freq_re_r[palavra]
        elif palavra not in freq_re_r:
            prob_frase_dado_r *= (alpha/(freq_ab_r.sum() + (alpha*v)))      
            
      
        if palavra in freq_re_i:
            prob_frase_dado_i *= freq_re_i[palavra]
        elif palavra not in freq_re_i:
            prob_frase_dado_i *= (alpha/(freq_ab_i.sum() + (alpha*v)))
            
      
       
  
    prob_r_dado_frase = prob_frase_dado_r*p_r

    prob_i_dado_frase = prob_frase_dado_i*p_i

    
   
    if max(log(prob_r_dado_frase, 10), log(prob_i_dado_frase, 10)) == log(prob_r_dado_frase, 10): 
        return 'Relevante'
    if max(log(prob_r_dado_frase, 10), log(prob_i_dado_frase, 10)) == log(prob_i_dado_frase, 10):
        return 'Irrelevante'
  

In [84]:
train['Classificacao'] = train['Treinamento'].apply(datawitter)

In [85]:
tabela = pd.crosstab(train['Relevancia'], train['Classificacao'], normalize='all')
tabela

Classificacao,Irrelevante,Relevante
Relevancia,,
Irrelevante,0.66,0.00
Relevante,0.03,0.31


In [86]:
f'{(tabela.iloc[0,0] + tabela.iloc[1,1])*100:5g}%'

'   97%'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [87]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test['Teste'] = test['Teste'].astype('str')
test['Teste'] = test['Teste'].apply(cleanup)
test['Teste'] = test['Teste'].str.lower()

In [88]:
test['Relevancia'] = test['Relevancia'].astype('category')
test['Relevancia'].cat.categories = [ 'Irrelevante', 'Relevante']

In [89]:
test['Classificador'] = test['Teste'].apply(datawitter)

In [90]:
tabela2 = pd.crosstab(test['Relevancia'], test['Classificador'], normalize='all')
tabela2

Classificador,Irrelevante,Relevante
Relevancia,,
Irrelevante,0.755,0.10
Relevante,0.055,0.09


In [91]:
f'O acerto do nosso classificador foi de: {(tabela2.iloc[0,0]+tabela2.iloc[1,1]+0)*100:.5g}%'

'O acerto do nosso classificador foi de: 84.5%'

___
### Concluindo

In [92]:
print(f'Porcentagem de mensagens relevantes que foram classificadas como relevantes: {(tabela2.iloc[1,1])*100:.5g}%')
print()
print(f'Porcentagem de mensagens relevantes que não foram classificadas como relevantes: {(1-tabela2.iloc[1,1])*100:.5g}%')

Porcentagem de mensagens relevantes que foram classificadas como relevantes: 9%

Porcentagem de mensagens relevantes que não foram classificadas como relevantes: 91%


In [93]:
print(f'Porcentagem de mensagens irrelevantes que foram classificadas como irrelevantes: {(tabela2.iloc[1,0])*100:.5g}%')
print()
print(f'Porcentagem de mensagens irrelevantes que não foram classificadas como irrelevantes: {(1-tabela2.iloc[1,0])*100:.5g}%')

Porcentagem de mensagens irrelevantes que foram classificadas como irrelevantes: 5.5%

Porcentagem de mensagens irrelevantes que não foram classificadas como irrelevantes: 94.5%


___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**